In [1]:
import pandas as pd
import numpy as np
import datetime
import glob

In [2]:
start="2009-10-14"

In [3]:
libor=pd.read_csv("LIBOR USD.csv")

In [4]:
libor["Date"]=libor["Date"].map(lambda x:pd.to_datetime(x,format='%d.%m.%Y'))

In [5]:
libor["Week"]=libor["Date"].map(lambda x:x.to_period("W"))

In [6]:
libor=libor.sort_values("Date")

In [7]:
libor=libor[libor["Date"]>start]
libor=libor.groupby("Week")["ON","1W","1M","2M","3M","6M","12M"].mean()

In [8]:
path_trea =r'./Treasury' 
allFiles_trea = glob.glob(path_trea + "/*.csv")
frame_trea = pd.DataFrame()
list_trea = []
for file_trea in allFiles_trea:
    df_trea = pd.read_csv(file_trea,index_col="Date", header=0)
    list_trea.append(df_trea)
frame_trea=pd.concat(list_trea)

In [9]:
frame_trea.index=frame_trea.index.map(lambda x:pd.to_datetime(x,format='%m/%d/%Y'))

In [10]:
frame_trea["Week"]=frame_trea.index.map(lambda x:x.to_period("W"))

In [11]:
frame_trea.to_csv("Treasury.csv")

In [12]:
frame_trea=frame_trea[frame_trea.index>start]
frame_trea=frame_trea.groupby("Week")["1 Mo","6 Mo","1 Yr","5 Yr","10 Yr","20 Yr","30 Yr"].mean()

In [13]:
hud=pd.read_csv("RM-A_09-30-2019.csv")

In [14]:
hud["DATE"]=hud["INITIAL ENDORSEMENT DATE"].map(lambda x:pd.to_datetime(x,format='%m/%d/%Y'))

In [15]:
hud["Week"]=hud["DATE"].map(lambda x:x.to_period("W"))

In [16]:
hud=hud[hud["DATE"]>start].reset_index()

In [17]:
cata=["202", "811"]
catb=["542(c)","542(b)"]
catc=["221","220","231","213"]
catd="223(f)"
cate="223(a)(7)"
catg="241a"
cath=["234(d)","207","223(d)"]
cati=["232","Board&Care","ICF","Nursing"]
catj=["Hospital","Group Practice"]

In [18]:
cats=[cata,catb,catc,catd,cate,catg,cath,cati,catj]

In [19]:
for i in range(len(cats)):
    name="Category %i"%(i)
    hud[name]=False
    for j in range(len(hud)):
        for k in range(len(cats[i])):
            if (cats[i][k] in hud["SOA CATEGORY/SUB CATEGORY"][j]):
                hud[name][j]=True

C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [20]:
hud["Category"]=None
for i in range(len(hud)):
    hud["Category"][i]=[]
    if hud["Category 0"][i]==True:
        hud["Category"][i].append("Capital Advances")
    if hud["Category 1"][i]==True:
        hud["Category"][i].append("Risk Sharing")
    if hud["Category 2"][i]==True:
        hud["Category"][i].append("New Construction")
    if hud["Category 3"][i]==True:
        hud["Category"][i].append("Refinancing_Purchase of Apartments_Coops")
    if hud["Category 4"][i]==True:
        hud["Category"][i].append("Refinancing of Apt")
    if hud["Category 5"][i]==True:
        hud["Category"][i].append("Improvements_Additions for Apartments_Coops")
    if hud["Category 6"][i]==True:
        hud["Category"][i].append("Other FHA")
    if hud["Category 7"][i]==True:
        hud["Category"][i].append("Health Care Facilities")
    if hud["Category 8"][i]==True:
        hud["Category"][i].append("Hospitals and Group Practices")

C:\Users\19018\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
hud.columns

Index(['index', 'HUD PROJECT NUMBER', 'PROPERTY NAME', 'PROPERTY STREET',
       'PROPERTY CITY', 'PROPERTY STATE', 'PROPERTY ZIP', 'UNITS',
       'INITIAL ENDORSEMENT DATE', 'FINAL ENDORSEMENT DATE',
       'ORIGINAL MORTGAGE AMOUNT', 'FIRST PAYMENT DATE', 'MATURITY DATE',
       'TERM IN MONTHS', 'INTEREST RATE', 'CURRENT PRINCIPAL AND INTEREST',
       'AMORTIZED PRINCIPAL BALANCE', 'HOLDER NAME', 'HOLDER CITY',
       'HOLDER STATE', 'SERVICER NAME', 'SERVICER CITY', 'SERVICER STATE',
       'SECTION OF ACT CODE', 'SOA CATEGORY/SUB CATEGORY', 'TE', 'TC', 'DATE',
       'Week', 'Category 0', 'Category 1', 'Category 2', 'Category 3',
       'Category 4', 'Category 5', 'Category 6', 'Category 7', 'Category 8',
       'Category'],
      dtype='object')

In [22]:
hud_cat=hud[['HUD PROJECT NUMBER', 'PROPERTY NAME', 'PROPERTY STREET',
       'PROPERTY CITY', 'PROPERTY STATE', 'PROPERTY ZIP', 'UNITS',
       'INITIAL ENDORSEMENT DATE', 'FINAL ENDORSEMENT DATE',
       'ORIGINAL MORTGAGE AMOUNT', 'FIRST PAYMENT DATE', 'MATURITY DATE',
       'TERM IN MONTHS', 'INTEREST RATE', 'CURRENT PRINCIPAL AND INTEREST',
       'AMORTIZED PRINCIPAL BALANCE', 'HOLDER NAME', 'HOLDER CITY',
       'HOLDER STATE', 'SERVICER NAME', 'SERVICER CITY', 'SERVICER STATE',
       'SECTION OF ACT CODE', 'SOA CATEGORY/SUB CATEGORY','Week','Category']]

In [126]:
hud_cat["PROPERTY CITY"]=hud_cat["PROPERTY CITY"].map(lambda x:x.upper())
hud_cat=hud_cat.sort_values("PROPERTY CITY")


In [127]:
hud_cat.to_csv("HUD with Category.csv")

In [25]:
len(hud["PROPERTY CITY"].unique())

3959

In [26]:
# capital advance
CA=hud[hud["Category 0"]==True].groupby(["PROPERTY CITY","Week"])["INTEREST RATE"].mean()

In [27]:
aa=hud.groupby(["PROPERTY CITY","Week"])["INTEREST RATE"].mean()
aa=aa.reset_index(["PROPERTY CITY","Week"])

In [28]:
aa.to_csv("City Data for Risk Sharing with HFAs & QPEs Type Loan.csv")

In [29]:
# Risk Sharing with HFAs & QPEs
RS=hud[hud["Category 1"]==True].groupby(["PROPERTY STATE","Week"])["INTEREST RATE"].mean()
RS=RS.reset_index(["PROPERTY STATE","Week"])

In [30]:
states=RS["PROPERTY STATE"].unique()
df_list=[]
df_s=pd.DataFrame()
for i in range(len(states)):
    df=RS[RS["PROPERTY STATE"]==states[i]]
    df=df.merge(libor,on="Week")
    df["spread between libor 12M"]=df["INTEREST RATE"]-df["12M"]
    df["Corr with libor 12M"]=df["12M"].corr(df["INTEREST RATE"])
    df=df.merge(frame_trea,on="Week")
    df["spread between 10 Yr treasury"]=df["INTEREST RATE"]-df["10 Yr"]
    df["Corr with 10 Yr treasury"]=df["10 Yr"].corr(df["INTEREST RATE"])
    df["spread between 20 Yr treasury"]=df["INTEREST RATE"]-df["20 Yr"]
    df["Corr with 20 Yr treasury"]=df["20 Yr"].corr(df["INTEREST RATE"])
    df["spread between 30 Yr treasury"]=df["INTEREST RATE"]-df["30 Yr"]
    df["Corr with 30 Yr treasury"]=df["30 Yr"].corr(df["INTEREST RATE"])
    df_list.append(df)
df_s=pd.concat(df_list).reset_index(drop=True)
    

C:\Users\19018\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\19018\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [31]:
df_s.to_csv("State Data for Risk Sharing with HFAs & QPEs Type Loan.csv")

In [32]:
hud=hud.groupby(["PROPERTY CITY","Week"])["INTEREST RATE"].mean()
hud=hud.reset_index(["PROPERTY CITY","Week"])

In [34]:
city=hud["PROPERTY CITY"].unique()
df_list=[]
df_s=pd.DataFrame()
for i in range(len(city)):
    df=hud[hud["PROPERTY CITY"]==city[i]]
    df=df.merge(libor["ON"],on="Week")
    df["spread between libor ON"]=df["INTEREST RATE"]-df["ON"]
    df["Corr with libor ON"]=df["ON"].corr(df["INTEREST RATE"])
    
    df=df.merge(libor["1W"],on="Week")
    df["spread between libor 1W"]=df["INTEREST RATE"]-df["1W"]
    df["Corr with libor 1W"]=df["1W"].corr(df["INTEREST RATE"])
    
    df=df.merge(libor["1M"],on="Week")
    df["spread between libor 1M"]=df["INTEREST RATE"]-df["1M"]
    df["Corr with libor 1M"]=df["1M"].corr(df["INTEREST RATE"])
    
    df=df.merge(libor["2M"],on="Week")
    df["spread between libor 2M"]=df["INTEREST RATE"]-df["2M"]
    df["Corr with libor 2M"]=df["2M"].corr(df["INTEREST RATE"])
    
    df=df.merge(libor["3M"],on="Week")
    df["spread between libor 3M"]=df["INTEREST RATE"]-df["3M"]
    df["Corr with libor 3M"]=df["3M"].corr(df["INTEREST RATE"])
    
    df=df.merge(libor["6M"],on="Week")
    df["spread between libor 6M"]=df["INTEREST RATE"]-df["6M"]
    df["Corr with libor 6M"]=df["6M"].corr(df["INTEREST RATE"])
    
    df=df.merge(libor["12M"],on="Week")
    df["spread between libor 12M"]=df["INTEREST RATE"]-df["12M"]
    df["Corr with libor 12M"]=df["12M"].corr(df["INTEREST RATE"])

    df=df.merge(frame_trea["1 Mo"],on="Week")
    df["spread between 1 Mo treasury"]=df["INTEREST RATE"]-df["1 Mo"]
    df["Corr with 1 Mo treasury"]=df["1 Mo"].corr(df["INTEREST RATE"])
    
    df=df.merge(frame_trea["6 Mo"],on="Week")
    df["spread between 6 Mo treasury"]=df["INTEREST RATE"]-df["6 Mo"]
    df["Corr with 6 Mo treasury"]=df["6 Mo"].corr(df["INTEREST RATE"])
    
    df=df.merge(frame_trea["1 Yr"],on="Week")
    df["spread between 1 Yr treasury"]=df["INTEREST RATE"]-df["1 Yr"]
    df["Corr with 1 Yr treasury"]=df["1 Yr"].corr(df["INTEREST RATE"])
    
    df=df.merge(frame_trea["5 Yr"],on="Week")
    df["spread between 5 Yr treasury"]=df["INTEREST RATE"]-df["5 Yr"]
    df["Corr with 5 Yr treasury"]=df["5 Yr"].corr(df["INTEREST RATE"])
    
    df=df.merge(frame_trea["10 Yr"],on="Week")
    df["spread between 10 Yr treasury"]=df["INTEREST RATE"]-df["10 Yr"]
    df["Corr with 10 Yr treasury"]=df["10 Yr"].corr(df["INTEREST RATE"])
    
    df=df.merge(frame_trea["20 Yr"],on="Week")
    df["spread between 20 Yr treasury"]=df["INTEREST RATE"]-df["20 Yr"]
    df["Corr with 20 Yr treasury"]=df["20 Yr"].corr(df["INTEREST RATE"])
    
    df=df.merge(frame_trea["30 Yr"],on="Week")
    df["spread between 30 Yr treasury"]=df["INTEREST RATE"]-df["30 Yr"]
    df["Corr with 30 Yr treasury"]=df["30 Yr"].corr(df["INTEREST RATE"])
    df_list.append(df)
df_s=pd.concat(df_list).reset_index(drop=True)


df_s.to_csv("City Data of HUD.csv",index=False)

In [129]:
categories=["Risk Sharing","New Construction","Refinancing_Purchase of Apartments_Coops",
     "Refinancing of Apt","Improvements_Additions for Apartments_Coops","Other FHA","Health Care Facilities",
     "Hospitals and Group Practices"]

for i in range(len(categories)):
    hud_sub=hud_cat[hud_cat["Category"].str.contains(categories[i],regex=False)]
    hud_sub=hud_sub.groupby(["PROPERTY CITY","Week"])["INTEREST RATE"].mean()
    hud_sub=hud_sub.reset_index(["PROPERTY CITY","Week"])
    hud_sub["Category"]=categories[i]
    city=hud_sub["PROPERTY CITY"].unique()
    df_list=[]
    df_s=pd.DataFrame()
    name="City Data of HUD_"+categories[i]
    for i in range(len(city)):
        df=hud_sub[hud_sub["PROPERTY CITY"]==city[i]]
        df=df.merge(libor["ON"],on="Week")
        df["spread between libor ON"]=df["INTEREST RATE"]-df["ON"]
        df["Corr with libor ON"]=df["ON"].corr(df["INTEREST RATE"])

        df=df.merge(libor["1W"],on="Week")
        df["spread between libor 1W"]=df["INTEREST RATE"]-df["1W"]
        df["Corr with libor 1W"]=df["1W"].corr(df["INTEREST RATE"])

        df=df.merge(libor["1M"],on="Week")
        df["spread between libor 1M"]=df["INTEREST RATE"]-df["1M"]
        df["Corr with libor 1M"]=df["1M"].corr(df["INTEREST RATE"])

        df=df.merge(libor["2M"],on="Week")
        df["spread between libor 2M"]=df["INTEREST RATE"]-df["2M"]
        df["Corr with libor 2M"]=df["2M"].corr(df["INTEREST RATE"])

        df=df.merge(libor["3M"],on="Week")
        df["spread between libor 3M"]=df["INTEREST RATE"]-df["3M"]
        df["Corr with libor 3M"]=df["3M"].corr(df["INTEREST RATE"])

        df=df.merge(libor["6M"],on="Week")
        df["spread between libor 6M"]=df["INTEREST RATE"]-df["6M"]
        df["Corr with libor 6M"]=df["6M"].corr(df["INTEREST RATE"])

        df=df.merge(libor["12M"],on="Week")
        df["spread between libor 12M"]=df["INTEREST RATE"]-df["12M"]
        df["Corr with libor 12M"]=df["12M"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["1 Mo"],on="Week")
        df["spread between 1 Mo treasury"]=df["INTEREST RATE"]-df["1 Mo"]
        df["Corr with 1 Mo treasury"]=df["1 Mo"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["6 Mo"],on="Week")
        df["spread between 6 Mo treasury"]=df["INTEREST RATE"]-df["6 Mo"]
        df["Corr with 6 Mo treasury"]=df["6 Mo"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["1 Yr"],on="Week")
        df["spread between 1 Yr treasury"]=df["INTEREST RATE"]-df["1 Yr"]
        df["Corr with 1 Yr treasury"]=df["1 Yr"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["5 Yr"],on="Week")
        df["spread between 5 Yr treasury"]=df["INTEREST RATE"]-df["5 Yr"]
        df["Corr with 5 Yr treasury"]=df["5 Yr"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["10 Yr"],on="Week")
        df["spread between 10 Yr treasury"]=df["INTEREST RATE"]-df["10 Yr"]
        df["Corr with 10 Yr treasury"]=df["10 Yr"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["20 Yr"],on="Week")
        df["spread between 20 Yr treasury"]=df["INTEREST RATE"]-df["20 Yr"]
        df["Corr with 20 Yr treasury"]=df["20 Yr"].corr(df["INTEREST RATE"])

        df=df.merge(frame_trea["30 Yr"],on="Week")
        df["spread between 30 Yr treasury"]=df["INTEREST RATE"]-df["30 Yr"]
        df["Corr with 30 Yr treasury"]=df["30 Yr"].corr(df["INTEREST RATE"])
        df_list.append(df)
    if len(df_list)!=0:
        df_s=pd.concat(df_list).reset_index(drop=True)
        df_s.to_csv(name+".csv",index=False)

In [12]:
cities=["BOSTON", "PHILADELPHIA", "NEW YORK", "WASHINGTON", "CHICAGO", "LOS ANGELES", "ATLANTA"]

In [13]:
hud_cat=pd.read_csv("HUD with Category.csv")

In [15]:
health=pd.read_csv("City Data of HUD_Health Care Facilities.csv")

In [16]:
health

,PROPERTY CITY,Week,INTEREST RATE,Category,ON,spread between libor ON,Corr with libor ON,1W,spread between libor 1W,Corr with libor 1W,...,Corr with 5 Yr treasury,10 Yr,spread between 10 Yr treasury,Corr with 10 Yr treasury,20 Yr,spread between 20 Yr treasury,Corr with 20 Yr treasury,30 Yr,spread between 30 Yr treasury,Corr with 30 Yr treasury
0,ABERCEN,2013-08-26/2013-09-01,4.89,Health Care Facilities,0.117150,4.772850,NaN,0.148330,4.741670,NaN,...,NaN,2.7640,2.1260,NaN,3.476,1.414,NaN,3.724,1.166,NaN
1,ABERDEEN,2013-06-03/2013-06-09,2.93,Health Care Facilities,0.133340,2.796660,NaN,0.158510,2.771490,NaN,...,NaN,2.1240,0.8060,NaN,2.928,0.002,NaN,3.276,-0.346,NaN
2,ABILENE,2013-11-11/2013-11-17,4.17,Health Care Facilities,0.102100,4.067900,NaN,0.128410,4.041590,NaN,...,NaN,2.7375,1.4325,NaN,3.525,0.645,NaN,3.820,0.350,NaN
3,ABINGDON,2018-04-23/2018-04-29,3.98,Health Care Facilities,1.703380,2.276620,NaN,1.747002,2.232998,NaN,...,NaN,2.9940,0.9860,NaN,3.072,0.908,NaN,3.170,0.810,NaN
4,ACTON,2012-12-17/2012-12-23,2.50,Health Care Facilities,0.163600,2.336400,NaN,0.187000,2.313000,NaN,...,NaN,1.8040,0.6960,NaN,2.558,-0.058,NaN,2.968,-0.468,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3421,YUMA,2015-10-26/2015-11-01,3.52,Health Care Facilities,0.130360,3.389640,-0.768248,0.154440,3.365560,-0.763337,...,-0.637098,2.1140,1.4060,0.806346,2.530,0.990,0.917387,2.898,0.622,0.956323
3422,YUMA,2017-11-13/2017-11-19,3.29,Health Care Facilities,1.182780,2.107220,-0.768248,1.203502,2.086498,-0.763337,...,-0.637098,2.3660,0.9240,0.806346,2.620,0.670,0.917387,2.814,0.476,0.956323
3423,ZABULON,2014-09-29/2014-10-05,3.67,Health Care Facilities,0.088300,3.581700,NaN,0.119340,3.550660,NaN,...,NaN,2.4660,1.2040,NaN,2.918,0.752,NaN,3.158,0.512,NaN
3424,ZANESVILLE,2018-09-10/2018-09-16,4.19,Health Care Facilities,1.917654,2.272346,NaN,1.961550,2.228450,NaN,...,NaN,2.9700,1.2200,NaN,3.046,1.144,NaN,3.114,1.076,NaN
